In [ ]:
import pickle

with open("patientTL.pickle",'rb') as handle:
    tweets = pickle.load(handle)

In [ ]:
print len(tweets)
print tweets[1]

In [ ]:
import unicodedata
from collections import defaultdict
user_tweets = defaultdict(list)
for tweet in tweets:
    text =  tweet[1]
    user_tweets[tweet[2]].append(text)

Hashtag analysis

In [ ]:
import re

list_with_tags = []
for i in range (0,len(tweets)):
    temp_list = re.findall(r'^#[a-z]{8,}(?=\s)|(?<=\s)#[a-z]{8,}(?=\s)|(?<=\s)#[a-z]{8,}$',tweets[i][1])
    if temp_list != []:
        list_with_tags.extend(temp_list)

collected_list = []
for j in range (0,len(list_with_tags)):
    temp_list = re.sub('#','',list_with_tags[j])
    collected_list.append(temp_list)

print "The number of hashtages I have collected is:"
print len(collected_list)
print collected_list[0]

In [ ]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import words

dictionary = set(words.words()) # words is a Python list
word_lemmatizer = WordNetLemmatizer()

def BackMaxMatch(strings):
    token = []
    pos_front = 0
    pos_end = len(strings)
    while len(strings) > 0:
        word = word_lemmatizer.lemmatize(strings[pos_front:pos_end])
        if word in dictionary:
            token.append(strings[pos_front:pos_end])
            strings = strings[0:pos_front]
            pos_end = len(strings)
            pos_front = 0
        else:
            if len(strings[pos_front:pos_end]) == 1:
                token.append(strings[pos_front:pos_end])
                strings = strings[0:pos_front]
                pos_end = len(strings)
                pos_front = 0
            else:
                pos_front = pos_front + 1
    token.reverse()
    return token

token_list_BMM = []
for i in range (0,len(collected_list)):
    temp_list = BackMaxMatch(collected_list[i])
    token_list_BMM.append(temp_list)

In [ ]:
def ForwardMaxMatch(strings):
    token = []
    pos_end = len(strings)
    while len(strings) > 0:
        word = word_lemmatizer.lemmatize(strings[:pos_end])
        if word in dictionary:
            token.append(strings[:pos_end])
            strings = strings[pos_end:]
            pos_end = len(strings)
        else:
            if len(strings[:pos_end]) == 1:
                token.append(strings[:pos_end])
                strings = strings[pos_end:]
                pos_end = len(strings)
            else:
                pos_end = pos_end - 1
    return token


token_list_FMM = []
for i in range (0,len(collected_list)):
    temp_list = ForwardMaxMatch(collected_list[i])
    token_list_FMM.append(temp_list)

In [ ]:
print "The different segmentations of hashtages by the BackMaxMatch(BMM) and ForwadMaxMatch(FMM) is:"

for j in range (0,len(collected_list)):
    if cmp(token_list_BMM[j],token_list_FMM[j]) != 0:
        print "Position of hashtage: " + str(j+1)
        print "#" + str(collected_list[j])
        print " The BMM result is: " + str(token_list_BMM[j])
        print " The FMM result is: " + str(token_list_FMM[j])
        
        if len(token_list_BMM[j]) <= len(token_list_FMM[j]):
            print " The best result is: " + str(token_list_BMM[j]) + '\n'
        else:
            print " The best result is: " + str(token_list_FMM[j]) + '\n'

Optimize the parameter

In [ ]:
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split

stop_words = set(stopwords.words('english'))
positive_tweets = nltk.corpus.twitter_samples.tokenized("positive_tweets.json")
negative_tweets = nltk.corpus.twitter_samples.tokenized("negative_tweets.json")

def text_cleaner(tweets):
    cleaned_token = []
    for i in range (0,len(tweets)):
        single_token = []
        for j in range(0,len(tweets[i])):
            if tweets[i][j] not in stop_words and tweets[i][j].isalpha():
                temp = tweets[i][j]
                if len(temp) != 0:
                    single_token.append(temp)
        cleaned_token.append(single_token)
    return cleaned_token

positive = text_cleaner(positive_tweets)
negative = text_cleaner(negative_tweets)

#First split the dataset as the 80% of train, 20% of rest. Then split the rest part into half develop and half test.
pos_train, pos_rest, neg_train, neg_rest = train_test_split(positive, negative, train_size = 0.8)
pos_develop, pos_test, neg_develop, neg_test = train_test_split(pos_rest, neg_rest, train_size = 0.5)

pos_train.extend(neg_train)
pos_develop.extend(neg_develop)
pos_test.extend(neg_test)

train_data = pos_train
develop_data = pos_develop
test_data = pos_test

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import f1_score

def get_BOW(text):
    BOW = {}
    for word in text:
        BOW[word] = BOW.get(word,0) + 1
    return BOW

# use the entire dataset to bulid a unified vector, this could ensure the subset feature matrix
# transformed in to same dimension for continues processing
vectorizer = DictVectorizer()
entire_dataset = train_data + develop_data + test_data  

vector_shape_unified = []
for i in range (0,len(entire_dataset)):
    temp = get_BOW(entire_dataset[i])
    vector_shape_unified.append(temp)
unified_set = vectorizer.fit_transform(vector_shape_unified)

def prepare_data(vectorizer,data,feature_extractor):
    feature_matrix = []
    label = []
    for j in range (0,len(data)):
        feature_dict = feature_extractor(data[j])
        feature_matrix.append(feature_dict)
        if j < (len(data)/2):
            label.append("positive")
        else:
            label.append("negative")
    dataset = vectorizer.transform(feature_matrix)
    return dataset,label

train_set,train_label = prepare_data(vectorizer,train_data,get_BOW)
develop_set,develop_label = prepare_data(vectorizer,develop_data,get_BOW)
test_set,test_label = prepare_data(vectorizer,test_data,get_BOW)


def prepare_data_test(vectorizer,data,feature_extractor):
    feature_matrix = []
    for j in range (0,len(data)):
        feature_dict = feature_extractor(data[j])
        feature_matrix.append(feature_dict)
    dataset = vectorizer.transform(feature_matrix)
    return dataset

test_set = prepare_data_test(vectorizer,test_data,get_BOW)

def NB_parameter_optimize(train,t_label,develop,d_label):
    print "In Naive Bayes Classifier we adjust the Laplace/Lidstone smoothing parameter (alpha):"
    temp_accuracy = 0
    val = 0.1
    while (val <= 3):
        NB_classifier = MultinomialNB(alpha = val, class_prior=None, fit_prior=True)
        NB_classifier.fit(train_set,train_label)
        predicted = NB_classifier.predict(develop_set)
        accuracy = accuracy_score(develop_label,predicted)
        print "   The alpha is " + str(val) + ", the accuracy is " + str(accuracy) + "."
        if temp_accuracy <= accuracy:
            optimal_NB_parameter = val
            temp_accuracy = accuracy 
        val = val + 0.1
    return optimal_NB_parameter

NB_best_parameter = NB_parameter_optimize(train_set,train_label,develop_set,develop_label)
print "The optimal value of the parameter for Naive-Bayes classifier is: " + str(NB_best_parameter) +'\n'


def LR_optimize(train,t_label,develop,d_label):
    print "In Logistic Regression classifier we adjust the regularization strength parameter (C):"
    temp_accuracy = 0
    val = 0.1
    while (val <= 3):
        LR_classifier = LogisticRegression(C = val)
        LR_classifier.fit(train_set,train_label)
        predicted = LR_classifier.predict(develop_set)
        accuracy = accuracy_score(develop_label,predicted)
        print "   The C is " + str(val) + ", the accuracy is " + str(accuracy) + "."
        if temp_accuracy <= accuracy:
            optimal_LR_parameter = val
            temp_accuracy = accuracy 
        val = val + 0.1
    return optimal_LR_parameter

LR_best_parameter = LR_optimize(train_set,train_label,develop_set,develop_label)
print "The optimal value of the parameter for Logistic Regression classifier is: " + str(LR_best_parameter)

In [ ]:
NB_classifier = MultinomialNB(alpha = NB_best_parameter, class_prior=None, fit_prior=True)
NB_classifier.fit(train_set,train_label)
NB_predicted = NB_classifier.predict(test_set)
NB_accuracy = accuracy_score(test_label,NB_predicted)
print "The accuracy of Naive-Bayes model is: " + str(NB_accuracy)
print "and the macroaveraged f-score is: " + str(f1_score(test_label, NB_predicted, average='macro')) + '\n'

LR_classifier = LogisticRegression(C = LR_best_parameter)
LR_classifier.fit(train_set,train_label)
LR_predicted = LR_classifier.predict(test_set)
LR_accuracy = accuracy_score(test_label,LR_predicted)
print "The accuracy of Logistic Regression model is: " + str(LR_accuracy)
print "and the macroaveraged f-score is: " + str(f1_score(test_label, LR_predicted, average='macro'))

Sentiment Analysis

In [ ]:
word_lemmatizer = WordNetLemmatizer()
word_tokenizer = nltk.tokenize.regexp.WordPunctTokenizer()
temp_stop_words = stopwords.words('english')
temp_stop_words.append(u'https')
temp_stop_words.append(u'co')
stop_words = set(temp_stop_words)


def text_tokenizer(tweets):
    tokenized_tweets = []
    for tweet in tweets:
        tokenized_tweet = word_tokenizer.tokenize(tweet)
        tokenized_tweets.append(tokenized_tweet)
    return tokenized_tweets


def text_cleaner(tweets):
    cleaned_token = []
    for i in range (0,len(tweets)):
        single_token = []
        for j in range(0,len(tweets[i])):
            word = word_lemmatizer.lemmatize(tweets[i][j])
            if word not in stop_words and tweets[i][j].isalpha():
                temp = tweets[i][j]
                if len(temp) != 0:
                    single_token.append(temp)
        cleaned_token.append(single_token)
    return cleaned_token

In [ ]:
def sentiment_analysis(test_tweets,best_NB, best_logistic):
    import nltk
    from nltk.corpus import stopwords
    from sklearn.model_selection import train_test_split

    stop_words = set(stopwords.words('english'))
    positive_tweets = nltk.corpus.twitter_samples.tokenized("positive_tweets.json")
    negative_tweets = nltk.corpus.twitter_samples.tokenized("negative_tweets.json")

    def text_cleaner(tweets):
        cleaned_token = []
        for i in range (0,len(tweets)):
            single_token = []
            for j in range(0,len(tweets[i])):
                if tweets[i][j] not in stop_words and tweets[i][j].isalpha():
                    temp = tweets[i][j]
                    if len(temp) != 0:
                        single_token.append(temp)
            cleaned_token.append(single_token)
        return cleaned_token

    positive = text_cleaner(positive_tweets)
    negative = text_cleaner(negative_tweets)

    #First split the dataset as the 80% of train, 20% of rest. Then split the rest part into half develop and half test.
    pos_train, pos_rest, neg_train, neg_rest = train_test_split(positive, negative, train_size = 0.8)
    pos_develop, pos_test, neg_develop, neg_test = train_test_split(pos_rest, neg_rest, train_size = 0.5)

    pos_train.extend(neg_train)
    pos_develop.extend(neg_develop)
    pos_test.extend(neg_test)

    train_data = pos_train
    develop_data = pos_develop
    test_data = pos_test
    
    from sklearn.naive_bayes import MultinomialNB
    from sklearn.linear_model import LogisticRegression
    from sklearn.feature_extraction import DictVectorizer
    from sklearn.metrics import accuracy_score, classification_report
    from sklearn.metrics import f1_score

    def get_BOW(text):
        BOW = {}
        for word in text:
            BOW[word] = BOW.get(word,0) + 1
        return BOW

    # use the entire dataset to bulid a unified vector, this could ensure the subset feature matrix
    # transformed in to same dimension for continues processing
    vectorizer = DictVectorizer()
    entire_dataset = train_data + develop_data + test_tweets   #change here

    vector_shape_unified = []
    for i in range (0,len(entire_dataset)):
        temp = get_BOW(entire_dataset[i])
        vector_shape_unified.append(temp)
    unified_set = vectorizer.fit_transform(vector_shape_unified)

    def prepare_data(vectorizer,data,feature_extractor):
        feature_matrix = []
        label = []
        for j in range (0,len(data)):
            feature_dict = feature_extractor(data[j])
            feature_matrix.append(feature_dict)
            if j < (len(data)/2):
                label.append("positive")
            else:
                label.append("negative")
        dataset = vectorizer.transform(feature_matrix)
        return dataset,label

    train_set,train_label = prepare_data(vectorizer,train_data,get_BOW)
    develop_set,develop_label = prepare_data(vectorizer,develop_data,get_BOW)
    #test_set,test_label = prepare_data(vectorizer,test_data,get_BOW)


    def prepare_data_test(vectorizer,data,feature_extractor):
        feature_matrix = []
        for j in range (0,len(data)):
            feature_dict = feature_extractor(data[j])
            feature_matrix.append(feature_dict)
        dataset = vectorizer.transform(feature_matrix)
        return dataset

    test_tweets_set = prepare_data_test(vectorizer,test_tweets,get_BOW)

    NB_best_parameter = best_NB

    LR_best_parameter = best_logistic
    
    NB_classifier = MultinomialNB(alpha = NB_best_parameter, class_prior=None, fit_prior=True)
    NB_classifier.fit(train_set,train_label)
    NB_predicted = NB_classifier.predict(test_tweets_set)
    
    positive_count = 0
    negative_count = 0
    for emotion in NB_predicted:
        if emotion == 'positive':
            positive_count += 1
        elif emotion == 'negative':
            negative_count += 1
    if positive_count > negative_count:
        return "pos"
    else:
        return "neg"

In [ ]:
sentiments = defaultdict(int)
i = 0
for key in user_tweets.keys():
    temp = user_tweets[key]
    temp_token = text_tokenizer(temp)
    test_tweets = text_cleaner(temp_token)
    sentiment = sentiment_analysis(test_tweets)
    sentiments[sentiment] +=1
    print i
    i +=1

In [ ]:
sentiments

In [ ]:
positive_count = 0
negative_count = 0
for emotion in NB_predicted:
    if emotion == 'positive':
        positive_count += 1
    elif emotion == 'negative':
        negative_count += 1

In [ ]:
print positive_count
print negative_count